In [24]:
import pandas as pd
import numpy as np
import gspread
from df2gspread import df2gspread as d2g
import random 
import pickle
import matplotlib.pyplot as plt
from random import randrange
import time

#### Import files

In [64]:
non_weight = ['cipolle',
              # 'peperoni gialli',
              'pomodoro conserva (sostanza secca 30%)',
              'brodo vegetale',
              'noce moscata',
              # 'zucchine',
              'carote',
              # 'spinaci',
              # 'piselli surgelati',
              # 'pomodori da insalata',
              # 'broccolo a testa',
              'alloro secco',
              'rosmarino secco',
              # 'peperoni dolci',
              'sedano',
              'prezzemolo fresco',
              'lamponi',
              'salvia fresca',
              # 'melanzane',
              'grappa',
              # 'zucca gialla',
              # 'asparagi di serra',
              # 'olive nere',
              'timo secco macinato',
              # 'fagiolini',
              'cipolline',
              # 'cavolfiore',
              'peperoncini piccanti',
              # 'rape',
              'paprika in polvere',
              'basilico fresco',
              'origano secco macinato',
              # 'finocchio',
              'succo di limone, fresco',
              # 'cavoli di bruxelles',
              # 'funghi prataioli , coltivati',
              'limone',
              "capperi sott'aceto",
              # 'broccoletti di rape',
              'olive da tavola, conservate',
              # 'radicchio rosso',
              'prezzemolo fresco',
              'vino bianco',
              # 'piselli surgelati',
              'noce moscata',
              'birra scura',
             ]

In [65]:
main_sources = {
    'random': {
        'dinner': [
            'vitello carne semigrassa',
            'suino carne semigrassa',
            'orata surgelata',
            "baccala' ammollato",
        ],
        'lunch': [
            'riso sbramato (integrale)',
        ],
        'universal':[
            'lenticchie secche',
            'orzo perlato',
            'fagioli secchi',
            'uovo di gallina, albume',
        ],
    },
    'stable': {
        'dinner': ['pollo petto'],
        'lunch': ['pasta proteica foodspring'],
    }
}

In [66]:
meal_plan = {
    'Energia - kcal': 2650,
    'Proteine totali': 182,
    'Lipidi totali': 117,
    'Colesterolo': 300,
    'Na': 2300,
    'Fibra alimentare ': 30,
}

meal_distribution = {
    'breakfast': 0.1,
    'lunch': 0.6,
    'veg_side': 0.05,
    'carb_side': 0.05,
    'universal': 0.6,
    'snack': 0.05,
    'dinner': 0.3,
}

In [67]:
how_many = pd.DataFrame()
for i in meal_distribution.keys():
    how_many[i] = pd.Series(meal_plan) * meal_distribution[i]

In [68]:
important_values = [
    'nome',
    'ingredient',
    'quantity', 
    'number_code',
    'Energia - kcal', 
    'Proteine totali',
    'Proteine animali', 
    'Proteine vegetali', 
    'Lipidi totali', 
    'Lipidi animali', 
    'Lipidi vegetali', 
    'Lipidi saturi', 
    'Colesterolo', 
    'Fibra alimentare ', 
    'Na']

In [69]:
rec_values = pd.read_pickle('pickle_files/rec_values.pkl')

In [70]:
rec_values['number_code'] = rec_values['number_code'].astype('str')

#### Select one good random breakfast, lunch and dinner

In [1010]:
def get_selection(meal_type_selector, value, day_type,
                  main_of_week = 'random',
                  how_many = how_many,
                  main_sources = main_sources, 
                  rec_values = rec_values,
                  important_values = important_values,
                  non_weight = non_weight):
    
    # does not work
    # assert ((meal_type_selector == 'universal')) & ((day_type == 'stable')), 'no stable for universal'
    
    if meal_type_selector in ['dinner', 'lunch']:
        
        if main_of_week == 'random':
            main_of_week = random.choices(main_sources[day_type][meal_type_selector], k=1)
            # print(f'the main of the week is: {main_of_week}')

        filt = rec_values[rec_values['type'] == meal_type_selector][important_values].groupby(
            'nome').agg(list).apply(lambda x: main_of_week[0] in x['ingredient'], axis = 1) 
        select_rec = rec_values[rec_values['type'] == meal_type_selector][important_values].groupby(
            'nome').agg(sum)[filt]
        
    else:
        
        select_rec = rec_values[rec_values['type'] == meal_type_selector][important_values].groupby(
            'nome').agg(sum)
        
    assert len(select_rec) != 0, f'there are no recipts with {main_of_week}'
    
    if meal_type_selector not in ['veg_side', 'carb_side']:
        empty_weights = rec_values[[True if i in non_weight else False for i in rec_values['ingredient']]]
        empty_sum = empty_weights[empty_weights['type'] == meal_type_selector][important_values].groupby(
            'nome').agg(sum)
    
        common = list(set(empty_sum.index) & set(select_rec.index))
        a = select_rec['quantity'].copy()
        select_rec.loc[common, 'quantity'] = select_rec.loc[common, 'quantity'] - empty_sum.loc[common, 'quantity']
        if (a - select_rec['quantity']).sum() == 0:
            print('no weight removed')
        # print(str((a - select_rec['quantity']).sum()) + ' weight removed')
    
    target = how_many[meal_type_selector][value]
    select_rec = select_rec.div(select_rec['quantity'], axis = 0)
    nec_quant = target / select_rec[value]
    select_rec = select_rec.multiply(nec_quant, axis = 0)

    return select_rec

In [541]:
def change_quantity(recipy, how_much):
    recipy.loc[num_values] = recipy.loc[num_values].multiply(how_much / recipy['quantity'])
    return recipy

In [593]:
def get_choice(select_rec, n_good_ones, stopper, rec_values = rec_values):
    
    # attach recipt id and sort by quantity
    select_mer = select_rec.merge(rec_values[['nome', 'main_recipy_code']].set_index('nome'), 
         left_index = True, right_index = True).drop_duplicates().sort_values('quantity')
    
    assert len(select_mer) > 0, 'select_mer as len == 0'
    
    # regulate stopper so it does not go on forever
    max_types = len(select_mer['main_recipy_code'].value_counts())
    if max_types < stopper:
        stopper = max_types

    # select minimum of recipts to choose from among the best 
    while len(np.unique(select_mer.iloc[:n_good_ones]['main_recipy_code'])) < stopper:
        n_good_ones += 1
        if n_good_ones > len(select_mer):
            break
    
    # get candidates
    candidates = select_mer.iloc[:n_good_ones]

    # while n_good_ones > len(candidates):
    #      n_good_ones -= 1
        
    # select a random one, pure randomly, could be more sophysticated 
    #     from random import randrange
    #     selected = randrange(len(candidates))
    
    assert len(candidates) > 0, 'there are no candidates'
    # print(selected)
    # print(candidates)
    # print(len(candidates))
    return candidates #.iloc[selected]

In [798]:
select_rec = get_selection('dinner', 'Proteine totali', 'stable')
select_mer = select_rec.merge(rec_values[['nome', 'main_recipy_code']].set_index('nome'), 
         left_index = True, right_index = True).drop_duplicates().sort_values('quantity')
max_types = len(select_mer['main_recipy_code'].value_counts())
if max_types < stopper:
    stopper = max_types
while len(np.unique(select_mer.iloc[:n_good_ones]['main_recipy_code'])) < stopper:
    n_good_ones += 1
    if n_good_ones > len(select_mer):
        break
candidates = select_mer.iloc[:n_good_ones]
candidates = candidates.apply(lambda x: reg_quantity(x, 'dinner'), axis = 1)

In [594]:
num_values = [
    'quantity', 
    'Energia - kcal', 
    'Proteine totali',
    'Proteine animali', 
    'Proteine vegetali', 
    'Lipidi totali', 
    'Lipidi animali', 
    'Lipidi vegetali', 
    'Lipidi saturi', 
    'Colesterolo', 
    'Fibra alimentare ', 
    'Na']

In [595]:
veg_sides = rec_values[rec_values['type'] == 'veg_side'].groupby('nome').agg(sum)[num_values].apply(
    lambda x: change_quantity(x, 100), axis = 1).sort_values('Energia - kcal', ascending = False)[:15]

In [596]:
# random * 2, universal * 2, stable * 4, 

In [816]:
def reg_quantity(chosen_meal, meal_type_selector, boundiary_quantities = boundiary_quantities):

    if chosen_meal['quantity'] > boundiary_quantities[meal_type_selector][1]:
        chosen_meal = change_quantity(chosen_meal, boundiary_quantities[meal_type_selector][1])
    if chosen_meal['quantity'] < boundiary_quantities[meal_type_selector][0]:
        chosen_meal = change_quantity(chosen_meal, boundiary_quantities[meal_type_selector][0])
        
    return chosen_meal

In [817]:
def_values = ['quantity', 'Energia - kcal', 'Proteine totali', 'Lipidi totali', 'Colesterolo', 'Fibra alimentare ', 'type']

In [835]:
def get_average(meal_type, day_type, boundiary_quantities = boundiary_quantities):
    select_rec = get_selection(meal_type, 'Proteine totali', day_type)
    
    # minimum number to start with
    n_good_ones = 6
    # stopper is the min num of different recipies that we will end up with to choose from
    stopper = 4

    # get meals
    chosen_meals = get_choice(select_rec, n_good_ones, stopper)
    
    # regulate quantity
    chosen_meals = chosen_meals.apply(lambda x: reg_quantity(x, meal_type, boundiary_quantities), axis = 1)
    
    return chosen_meals.quantile(0.25)

In [836]:
frutta_index = [i for i in rec_values['nome'].value_counts().index if (i[:6] == 'frutta') & (len(i) < 10)]
average_fruit = get_selection('snack', 'Proteine totali', 'random').loc[frutta_index].dropna(0).apply(
    lambda x: change_quantity(x, 200), axis = 1).mean()

In [837]:
select_rec = get_selection('veg_side', 'Proteine totali', day_type)
# minimum number to start with
n_good_ones = 6
# stopper is the min num of different recipies that we will end up with to choose from
stopper = 4
chosen_meals = get_choice(select_rec, n_good_ones, stopper)
average_veg_side = chosen_meals.apply(lambda x: change_quantity(x, 200), axis = 1).mean()

In [849]:
select_rec = get_selection('carb_side', 'Proteine totali', day_type)
# minimum number to start with
n_good_ones = 6
# stopper is the min num of different recipies that we will end up with to choose from
stopper = 4
chosen_meals = get_choice(select_rec, n_good_ones, stopper)
average_carb_side = chosen_meals.apply(lambda x: change_quantity(x, 200), axis = 1).mean()

In [850]:
shake = rec_values[rec_values['nome'] == 'shake proteico'][num_values].sum()
shake = change_quantity(shake, 150)

In [851]:
boundiary_quantities = {
    'breakfast': [150, 200],
    'lunch': [200, 250],
    'dinner': [150, 200],
    'universal': [200, 250],
    'snack': [80, 100],
}

In [862]:
total_week = sum(
    [
        # dinner 
        get_average('dinner', 'stable', boundiary_quantities).loc[['Energia - kcal', 'Proteine totali']] * 3,
        get_average('dinner', 'random', boundiary_quantities).loc[['Energia - kcal', 'Proteine totali']] * 2,
        
        # lunch
        get_average('lunch', 'stable', boundiary_quantities).loc[['Energia - kcal', 'Proteine totali']] * 3,
        get_average('lunch', 'random', boundiary_quantities).loc[['Energia - kcal', 'Proteine totali']] * 0,
        
        # universal
        get_average('universal', 'stable', boundiary_quantities).loc[['Energia - kcal', 'Proteine totali']] * 2,
        
        # breakfast
        get_average('breakfast', 'stable', boundiary_quantities).loc[['Energia - kcal', 'Proteine totali']] * 5,
        
        # snack
        get_average('snack', 'random', boundiary_quantities).loc[['Energia - kcal', 'Proteine totali']] * 5,
        
        # side veg
        average_veg_side.loc[['Energia - kcal', 'Proteine totali']] * 5,
        
        # carb side
        average_carb_side.loc[['Energia - kcal', 'Proteine totali']] * 5,
        
        # fruit
        average_fruit.loc[['Energia - kcal', 'Proteine totali']] * 5,
        
        # shake
        shake.loc[['Energia - kcal', 'Proteine totali']] * 5, 
    ]
)
week_goal = pd.Series(meal_plan)*5
(total_week - week_goal).dropna()

Energia - kcal     14.752928
Proteine totali   -25.986583
dtype: float64

#### Upload select_rec

In [247]:
from oauth2client.service_account import ServiceAccountCredentials

scopes = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'credentials.json', scopes = scopes)

gc = gspread.authorize(credentials)

In [248]:
gc = gspread.service_account(filename='credentials.json')
sh = gc.open("recipy_review")
sheet_code = '1qjjXi7KLTe_MOV4b_AOI35mmuADD5z7nUV2osbNYLI4'

In [1000]:
all_random_dinners = [
    'vitello carne semigrassa',
    'suino carne semigrassa',
    'orata surgelata',
    "baccala' ammollato",
]

In [999]:
combs = [
    ['dinner', 'stable'],
    # ['dinner', 'random'],
    ['lunch', 'stable'],
    ['lunch', 'random'],
    ['universal', 'stable'],
    ['breakfast', 'stable'],
    ['snack', 'random'],
    ]

In [1012]:
sheet_name = 'chosen recipies'
space = 0
for idx, i in enumerate(combs):
    select_rec = get_selection(i[0], 'Proteine totali', i[1])
    
    # minimum number to start with
    n_good_ones = 6
    # stopper is the min num of different recipies that we will end up with to choose from
    stopper = 4

    # get meals
    chosen_meals = get_choice(select_rec, n_good_ones, stopper)
    # regulate quantity
    chosen_meals = chosen_meals.apply(lambda x: reg_quantity(x, i[0]), axis = 1)
    chosen_meals['type'] = (str(i[0]) + ' ' + str(i[1]))
    chosen_meals['prot_ points'] = chosen_meals['Proteine totali'] > chosen_meals['Proteine totali'].quantile(
        0.25)
    chosen_meals['eneg_ points'] = chosen_meals['Energia - kcal'] > chosen_meals['Energia - kcal'].quantile(
        0.25)
    to_upl = chosen_meals 
    
    print(f'Updating {i[0]} .... ' + ' '*10, end = '\r', flush = True)
    
    if idx == 0:
        d2g.upload(to_upl.round(1), sheet_code, sheet_name, credentials=credentials, row_names=True, 
               start_cell = f'A{1 + space}', clean = True)
    else:
        d2g.upload(to_upl.round(1), sheet_code, sheet_name, credentials=credentials, row_names=True, 
               start_cell = f'A{1 + space}', clean = False)
    
    worksheet = sh.worksheet(sheet_name)
    worksheet.update(f'A{1 + space}', i[0] + ' '+ i[1])
    worksheet.format(f"A{1 + space}:A{1 + space}", {
        "backgroundColor": {
          "red": 2.99,
          "green": 0.5,
          "blue": 0.5
        },
        "textFormat": {
          "fontSize": 11,
          "bold": True
        }
    })
    
    space += to_upl.shape[0] + 3
    
    n = 5
    for i in range(n):
        print(f'I am waiting {n-(i+1)} seconds .... '+ ' '*10, end = '\r', flush = True)
        time.sleep(1)
    
    

In [1013]:
for dinner in all_random_dinners:
    select_rec = get_selection('dinner', 'Proteine totali', 'random', [dinner])
    # minimum number to start with
    n_good_ones = 6
    # stopper is the min num of different recipies that we will end up with to choose from
    stopper = 4

    # get meals
    chosen_meals = get_choice(select_rec, n_good_ones, stopper)
    # regulate quantity
    chosen_meals = chosen_meals.apply(lambda x: reg_quantity(x, 'dinner'), axis = 1)
    chosen_meals['type'] = (str('dinner') + ' ' + str('random'))
    chosen_meals['prot_ points'] = chosen_meals['Proteine totali'] > chosen_meals['Proteine totali'].quantile(
        0.25)
    chosen_meals['eneg_ points'] = chosen_meals['Energia - kcal'] > chosen_meals['Energia - kcal'].quantile(
        0.25)
    to_upl = chosen_meals 
    
    print(f'Updating {dinner} .... ' + ' '*10, end = '\r', flush = True)
    
    if idx == 0:
        d2g.upload(to_upl.round(1), sheet_code, sheet_name, credentials=credentials, row_names=True, 
               start_cell = f'A{1 + space}', clean = True)
    else:
        d2g.upload(to_upl.round(1), sheet_code, sheet_name, credentials=credentials, row_names=True, 
               start_cell = f'A{1 + space}', clean = False)
    
    worksheet = sh.worksheet(sheet_name)
    worksheet.update(f'A{1 + space}', dinner)
    worksheet.format(f"A{1 + space}:A{1 + space}", {
        "backgroundColor": {
          "red": 2.99,
          "green": 0.5,
          "blue": 0.5
        },
        "textFormat": {
          "fontSize": 11,
          "bold": True
        }
    })
    
    space += to_upl.shape[0] + 3
    
    n = 5
    for i in range(n):
        print(f'I am waiting {n-(i+1)} seconds .... '+ ' '*10, end = '\r', flush = True)
        time.sleep(1)

In [1014]:
frutta_index = [i for i in rec_values['nome'].value_counts().index if (i[:6] == 'frutta') & (len(i) < 10)]
fruits = get_selection('snack', 'Proteine totali', 'random').loc[frutta_index].dropna(0).apply(
    lambda x: change_quantity(x, 200), axis = 1) 

select_rec = get_selection('veg_side', 'Proteine totali', day_type)
# minimum number to start with
n_good_ones = 6
# stopper is the min num of different recipies that we will end up with to choose from
stopper = 4
chosen_meals = get_choice(select_rec, n_good_ones, stopper)
veg_sides = chosen_meals.apply(lambda x: change_quantity(x, 200), axis = 1)

select_rec = get_selection('carb_side', 'Proteine totali', day_type)
# minimum number to start with
n_good_ones = 6
# stopper is the min num of different recipies that we will end up with to choose from
stopper = 4
chosen_meals = get_choice(select_rec, n_good_ones, stopper)
carb_sides = chosen_meals.apply(lambda x: change_quantity(x, 200), axis = 1)

In [1015]:
for i in zip(['fruits', 'veggies', 'extra carbs'], [fruits, veg_sides, carb_sides]):
    print(f'Updating {i[0]} .... ' + ' '*10, end = '\r', flush = True)
    
    d2g.upload(i[1].round(1), sheet_code, sheet_name, credentials=credentials, row_names=True, 
           start_cell = f'A{1 + space}', clean = False)
    
    worksheet = sh.worksheet(sheet_name)
    worksheet.update(f'A{1 + space}', i[0])
    worksheet.format(f"A{1 + space}:A{1 + space}", {
        "backgroundColor": {
          "red": 2.99,
          "green": 0.5,
          "blue": 0.5
        },
        "textFormat": {
          "fontSize": 11,
          "bold": True
        }
    })
    
    space += i[1].shape[0] + 3
    
    n = 5
    for i in range(n):
        print(f'I am waiting {n-(i+1)} seconds .... '+ ' '*10, end = '\r', flush = True)
        time.sleep(1)

#### print ingredient list 

In [921]:
from oauth2client.service_account import ServiceAccountCredentials

scopes = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'credentials.json', scopes = scopes)

gc = gspread.authorize(credentials)

In [922]:
gc = gspread.service_account(filename='credentials.json')
sh = gc.open("recipy_review")
sheet_code = '1qjjXi7KLTe_MOV4b_AOI35mmuADD5z7nUV2osbNYLI4'

In [936]:
sheet = pd.DataFrame(sh.sheet1.get_all_values())
sheet.columns = sheet.iloc[0]
sheet = sheet.iloc[1:]
sheet['portion'] = sheet['portion'].astype('float')
sheet['times'] = sheet['times'].astype('int')

In [995]:
shopping_group = pd.DataFrame({})
for idx, rec in sheet.iterrows():
    ings = rec_values[rec_values['nome'] == rec['recipy']]
    pure_ings = ings[[True if i not in non_weight else False for i in ings['ingredient']]][important_values]
    pure_ings[num_values] = pure_ings[num_values].multiply(rec['portion'] / pure_ings['quantity'].sum()) * rec['times']
    shopping_group = shopping_group.append(pure_ings)

In [998]:
shopping_list = shopping_group[['nome', 'ingredient', 'quantity']].groupby('ingredient').agg(sum)

gc = gspread.service_account(filename='credentials.json')
sh = gc.open("recipy_review")
sheet_code = '1qjjXi7KLTe_MOV4b_AOI35mmuADD5z7nUV2osbNYLI4'

d2g.upload(shopping_list.round(1), sheet_code, 'shopping list', credentials=credentials, row_names=True, clean = True)

<Worksheet 'shopping list' id:91385844>

In [923]:
for i in to_find_ing:
    rec_values[rec_values['nome'] == i]

In [881]:
time.sleep(5)

In [143]:
# day meals
day_meals = pd.DataFrame({})
# type of meal 
for meal_type_selector, day_type in zip(['dinner', 'lunch', 'breakfast'], ['random', 'random', 'stable']):
    # choose type of nutrient value to cap
    value = 'Proteine totali'
    # choose type of ingredient 
    # day_type = 'random' # stable or random
    # select recipts for that particular meal with that specific ingredient with the protein quantity right
    select_rec = get_selection(meal_type_selector, value, day_type)

    # minimum number to start with
    n_good_ones = 6
    # stopper is the min num of different recipies that we will end up with to choose from
    stopper = 4

    # get meals
    chosen_meals = get_choice(select_rec, n_good_ones, stopper)
    
    # regulate quantities
    chosen_meals = chosen_meals.apply(lambda x: reg_quantity(x), axis = 1)
    chosen_meals['type'] = meal_type_selector
    
    # select three meals in an intelligent way
    day_meals = day_meals.append(chosen_meals)

In [198]:
a = day_meals[day_meals['type'] == 'dinner']
b = day_meals[day_meals['type'] == 'lunch']
c = day_meals[day_meals['type'] == 'breakfast']
total_cal = 0 
counter = 0
t = how_many.loc['Energia - kcal'][['breakfast', 'lunch', 'dinner']].sum()
t2 = 0
result = pd.DataFrame({})
while True:
    counter += 1
    for i in [a, b, c]:
        selected = randrange(len(i))
        total_cal += i.iloc[selected]['Energia - kcal']
        result = result.append(i.iloc[selected])
    if total_cal > (t - t2):
        break
    else:
        total_cal = 0
        result = pd.DataFrame({})
    if counter > 50:
        if t2 > 500:
            print('no combination found')
            break
        else:
            print('reduction')
            counter = 0
            t2 += 100

reduction
reduction
reduction
reduction
reduction
reduction
no combination found


In [199]:
t

2650.0

In [200]:
t2

600

In [201]:
result.sum()

Series([], dtype: float64)

In [111]:
a = rec_values[rec_values['nome'] == 'risotto e piselli variante margarina vegetale'][num_values].sum()
change_quantity(a, 900)

quantity              900.000000
Energia - kcal       2135.132662
Proteine totali        55.399817
Proteine animali       23.385178
Proteine vegetali      32.014639
Lipidi totali         117.881061
Lipidi animali         16.468435
Lipidi vegetali       101.412626
Lipidi saturi          30.833852
Colesterolo            55.992681
Fibra alimentare       29.741995
Na                   2326.166514
dtype: float64

In [115]:
day_type = 'random'
meal_type_selector = 'lunch'
    
main_of_week = random.choices(main_sources[day_type][meal_type_selector], k=1)
# print(f'the main of the week is: {main_of_week}')

filt = rec_values[rec_values['type'] == meal_type_selector][important_values].groupby(
    'nome').agg(list).apply(lambda x: main_of_week[0] in x['ingredient'], axis = 1) 
select_rec = rec_values[rec_values['type'] == meal_type_selector][important_values].groupby(
    'nome').agg(sum)[filt]

empty_weights = rec_values[[True if i in non_weight else False for i in rec_values['ingredient']]]
empty_sum = empty_weights[empty_weights['type'] == meal_type_selector][important_values].groupby(
    'nome').agg(sum)

print(select_rec.loc['risotto con asparagi variante margarina vegetale'])

common = list(set(empty_sum.index) & set(select_rec.index))
a = select_rec['quantity'].copy()
select_rec.loc[common, 'quantity'] = select_rec.loc[common, 'quantity'] - empty_sum.loc[common, 'quantity']
if (a - select_rec['quantity']).sum() == 0:
    print('no weight removed')

print(select_rec.loc['risotto con asparagi variante margarina vegetale'])

target = how_many[meal_type_selector][value]
select_rec = select_rec.div(select_rec['quantity'], axis = 0)
nec_quant = target / select_rec[value]
select_rec = select_rec.multiply(nec_quant, axis = 0)

print(select_rec.loc['risotto con asparagi variante margarina vegetale'])

change_quantity(select_rec.loc['risotto con asparagi variante margarina vegetale'], 250)

quantity              521.500
Energia - kcal       1078.400
Proteine totali        26.040
Proteine animali       10.650
Proteine vegetali      15.390
Lipidi totali          53.630
Lipidi animali          7.500
Lipidi vegetali        46.130
Lipidi saturi          14.454
Colesterolo            25.500
Fibra alimentare        9.510
Na                   1366.400
Name: risotto con asparagi variante margarina vegetale, dtype: float64
quantity              381.500
Energia - kcal       1078.400
Proteine totali        26.040
Proteine animali       10.650
Proteine vegetali      15.390
Lipidi totali          53.630
Lipidi animali          7.500
Lipidi vegetali        46.130
Lipidi saturi          14.454
Colesterolo            25.500
Fibra alimentare        9.510
Na                   1366.400
Name: risotto con asparagi variante margarina vegetale, dtype: float64
quantity             1599.838710
Energia - kcal       4522.322581
Proteine totali       109.200000
Proteine animali       44.661290
Protei

quantity             250.000000
Energia - kcal       706.684142
Proteine totali       17.064220
Proteine animali       6.979030
Proteine vegetali     10.085190
Lipidi totali         35.144168
Lipidi animali         4.914810
Lipidi vegetali       30.229358
Lipidi saturi          9.471822
Colesterolo           16.710354
Fibra alimentare       6.231979
Na                   895.412844
Name: risotto con asparagi variante margarina vegetale, dtype: float64

In [131]:
b = get_selection('lunch', 'Proteine totali', 'random').apply(lambda x: change_quantity(x, 250), axis = 1)
b.loc['risotto con asparagi variante margarina vegetale']

c = get_choice(get_selection('lunch', 'Proteine totali', 'random'), 6, 4).apply(lambda x: change_quantity(x, 250), axis = 1)
c.loc['risotto e piselli variante margarina vegetale']

quantity             250.000000
Energia - kcal       797.355474
Proteine totali       20.688807
Proteine animali       8.733087
Proteine vegetali     11.955720
Lipidi totali         44.022140
Lipidi animali         6.150062
Lipidi vegetali       37.872079
Lipidi saturi         11.514760
Colesterolo           20.910209
Fibra alimentare      11.107011
Na                   868.696187
Name: risotto e piselli variante margarina vegetale, dtype: float64

In [116]:
day_meals.loc['risotto e piselli variante margarina vegetale']

Colesterolo          20.9102
Energia - kcal       797.355
Fibra alimentare      11.107
Lipidi animali       6.15006
Lipidi saturi        11.5148
Lipidi totali        44.0221
Lipidi vegetali      37.8721
Na                   868.696
Proteine animali     8.73309
Proteine totali      20.6888
Proteine vegetali    11.9557
main_recipy_code         NaN
quantity                 250
type                     NaN
Name: risotto e piselli variante margarina vegetale, dtype: object

In [103]:
select_rec

,quantity,Energia - kcal,Proteine totali,Proteine animali,Proteine vegetali,Lipidi totali,Lipidi animali,Lipidi vegetali,Lipidi saturi,Colesterolo,Fibra alimentare,Na
nome,,,,,,,,,,,,
"riso alla cinese variante margarina vegetale variante uovo di gallina, albume",1241.800629,3293.897620,109.2,61.538392,47.661608,175.494746,24.517288,150.977458,45.903718,83.358779,44.278222,3928.895375
risotto ai funghi variante margarina vegetale,1667.058824,4686.117647,109.2,46.537815,62.662185,236.315966,32.773109,203.542857,63.684706,111.428571,38.934454,5990.487395
risotto con asparagi variante margarina vegetale,1599.838710,4522.322581,109.2,44.661290,64.538710,224.900000,31.451613,193.448387,60.613548,106.935484,39.880645,5730.064516
risotto con seppioline alla veneziana variante margarina vegetale,966.360473,2974.819763,109.2,80.551148,28.648852,137.139596,20.517745,116.621851,36.980070,482.546973,17.528740,3829.725818
risotto di zucchine variante margarina vegetale,1581.057692,4430.192308,109.2,57.346154,51.853846,226.678846,40.384615,186.294231,65.035385,137.307692,33.559615,5692.211538
risotto e piselli variante margarina vegetale,1319.554102,4208.614744,109.2,46.095125,63.104875,232.358383,32.461356,199.897027,60.777396,110.368609,58.625208,4585.166468
risotto gongorzola e noci variante margarina vegetale,884.827109,4011.551867,109.2,65.877455,43.322545,261.596680,81.711203,179.885477,71.923983,239.394191,22.252835,6354.141079
"risotto gongorzola, noci e speck variante margarina vegetale",753.002676,3193.783408,109.2,80.287814,28.912186,202.200214,81.593149,120.607065,57.891586,273.535772,15.410740,7851.684567
risotto radicchio gorgoznola variante margarina vegetale,1324.658786,3852.806018,109.2,71.088555,38.111445,221.421924,86.080817,135.341107,73.377002,253.489522,31.862225,7008.809242


In [99]:
empty_weights[empty_weights['nome'] == 'risotto con asparagi variante margarina vegetale'].sum()['Energia - kcal']

19.6

In [ ]:
if meal_type_selector not in ['veg_side', 'carb_side']:
    empty_weights = rec_values[[True if i in non_weight else False for i in rec_values['ingredient']]]
    empty_sum = empty_weights[empty_weights['type'] == meal_type_selector][important_values].groupby(
        'nome').agg(sum)

    common = list(set(empty_sum.index) & set(select_rec.index))
    a = select_rec['quantity'].copy()
    select_rec.loc[common, 'quantity'] = select_rec.loc[common, 'quantity'] - empty_sum.loc[common, 'quantity']
    if (a - select_rec['quantity']).sum() == 0:
        print('no weight removed')
    # print(str((a - select_rec['quantity']).sum()) + ' weight removed')

target = how_many[meal_type_selector][value]
select_rec = select_rec.div(select_rec['quantity'], axis = 0)
nec_quant = target / select_rec[value]
select_rec = select_rec.multiply(nec_quant, axis = 0)

In [42]:
day_meals[['quantity', 'Energia - kcal', 'Proteine totali']]

,quantity,Energia - kcal,Proteine totali
nome,,,
carne alla pizzaiola,261.330063,822.864471,54.6
braciola di maiale variante margarina vegetale,324.775862,788.606897,54.6
filetto di vitello alla piastra variante margarina vegetale,324.775862,788.606897,54.6
spezzatino di carne,341.852352,553.823487,54.6
agnello brucialingua,358.724011,665.682785,54.6
arrosto di latte variante margarina vegetale,389.910591,876.078404,54.6
"risotto gongorzola, noci e speck variante margarina vegetale",753.002676,3193.783408,109.2
risotto e piselli variante margarina vegetale,832.633769,4208.614744,109.2
"riso alla cinese variante margarina vegetale variante uovo di gallina, albume",874.041311,3293.897620,109.2


In [ ]:
    

    
    day_meals = day_meals.append(chosen_meal)

# add one vegetarian side 
selected = randrange(len(veg_sides))
day_meals = day_meals.append(veg_sides.iloc[selected])
    
# calculate loss of values
loss = day_meals.sum()[how_many.index] - how_many[['dinner', 'lunch', 'breakfast']].sum(axis = 1)
print(loss[['Energia - kcal', 'Proteine totali']])
day_meals[day_meals.columns[::-1]]

In [ ]:
change_quantity(rec_values[rec_values['nome'] == 'shake proteico rinforzato'].sum()[num_values], 200)

In [ ]:
def create_base_3(rec_values = rec_values, 
                  value = value, 
                  day_type = day_type, 
                  n_good_ones = n_good_ones,
                  stopper = stopper):

    day_meals = {
        'dinner': '',
        'lunch': '',
        'breakfast': '',
    }
  
    # choose type of meal
    for meal_type_selector in ['dinner', 'lunch', 'breakfast']:
        # select recipts for that particular meal with that specific ingredient with the protein quantity right
        select_rec = get_selection(meal_type_selector, value, day_type)
        # choose randomly 
        random_rec = get_choice(select_rec, n_good_ones, stopper)
        random_rec['meal_type'] = meal_type_selector
        day_meals[random_rec.name] = random_rec
    day_meals = pd.DataFrame(day_meals)
    day_meals = day_meals.drop(['dinner', 'lunch', 'breakfast'], axis = 1).T
    
    return day_meals

#### Estimate average loss

In [ ]:
# choose type of nutrient value to cap
value = 'Proteine totali'
# choose type of ingredient 
day_type = 'random' # stable or random
# minimum number to start with
n_good_ones = 6
# stopper is the min num of different recipies that we will end up with to choose from
stopper = 4

loss_list = pd.DataFrame({})

for i in range(50):
    # print(str(i) + ' '*10, flush = True, end = '\r')
    day_meals = create_base_3()
    loss = day_meals.sum()[how_many.index] - how_many[day_meals_names.keys()].sum(axis = 1)
    loss.name = i
    loss_list = loss_list.append(loss)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (16, 8))
loss_list.plot(ax = ax)

In [ ]:
loss_list.mean()

#### Calculate stable day meal

In [ ]:
# choose type of nutrient value to cap
value = 'Proteine totali'
# choose type of ingredient 
day_type = 'stable'
# minimum number to start with
n_good_ones = 6
# stopper is the min num of different recipies that we will end up with to choose from
stopper = 4

loss_list = pd.DataFrame({})
# get meals
day_meals = create_base_3()
# calculate loss of values
loss = day_meals.sum()[how_many.index] - how_many[day_meals_names.keys()].sum(axis = 1)
loss.name = i
# get reaminder to compensate
remainer = ((how_many['side'] * 2) - loss)
remainer.loc[remainer < 0] = 0
# select snacks 
meal_type_selector = 'snack'
value = 'Proteine totali'
select_rec = rec_values[rec_values['type'] == meal_type_selector][important_values].groupby(
    'nome').agg(sum)

empty_weights = rec_values[[True if i in non_weight else False for i in rec_values['ingredient']]]
empty_sum = empty_weights[empty_weights['type'] == meal_type_selector][important_values].groupby(
    'nome').agg(sum)

common = list(set(empty_sum.index) & set(select_rec.index))
a = select_rec['quantity'].copy()
select_rec.loc[common, 'quantity'] = select_rec.loc[common, 'quantity'] - empty_sum.loc[common, 'quantity']
if (a - select_rec['quantity']).sum() == 0:
    print('no weight removed')
# get fruits indexes 
frutta_index = [i for i in select_rec.index if (i[:6] == 'frutta') & (len(i) < 10)]

In [ ]:
day_meals

In [ ]:
def get_n_values(select_rec, how_much, value):
    return select_rec.multiply(how_much / select_rec[value], axis = 0).sort_values('quantity')

In [ ]:
# get first snack 
snack_1_cand = (remainer - select_rec[remainer.index]).sort_values('Energia - kcal').iloc[:10].index
# get candiates snacks
candi_snack = get_n_values(select_rec.loc[snack_1_cand], (remainer['Energia - kcal'] - 100), 'Energia - kcal')
# select using function of before
snack_of_choice = get_choice(candi_snack, 6, 6)

In [ ]:
# get 100 kcals of fruit
fruit_quant = get_n_values(select_rec.loc[frutta_index], 100, 'Energia - kcal')[1:15]['quantity']
# get free fruit for the last 100 kcals
free_fruit = rec_values[['ingredient', 'nome']].set_index('nome').loc[frutta_index].merge(
    fruit_quant, left_index = True, right_index = True)

In [ ]:
def get_ingredient_list(food_values, food_name, quantity):
    ing = rec_values[rec_values['nome'] == food_name]
    mult = quantity / ing['quantity'].sum()
    ing[food_values.index[:-1]] = (ing[food_values.index[:-1]] * mult).astype('int')
    return ing

In [ ]:
get_ingredient_list(snack_of_choice, snack_of_choice.name, snack_of_choice.quantity)[important_values]

In [ ]:
l = pd.DataFrame({})
for idx, row in day_meals.iterrows():
    l = l.append(get_ingredient_list(row[:-2], row.name, row.quantity))
l[l['type'] == 'lunch']

In [ ]:
food_name = snack_of_choice.name
quantity = snack_of_choice.quantity
food_values = snack_of_choice
ing = rec_values[rec_values['nome'] == food_name]
# print(ing)
mult = quantity / ing['quantity'].sum()
ing[food_values.index[:-1]] = (ing[food_values.index[:-1]] * mult).astype('int')
ing.sum() #[['quantity', 'ingredient', 'nome'][::-1]]

In [ ]:
snack_of_choice

In [ ]:
get_ingredient_list(snack_of_choice, snack_of_choice.name, snack_of_choice.quantity)

In [ ]:
snack_of_choice.index

In [ ]:
free_fruit.iloc[0]